In [4]:
import pandas as pd
import numpy as np
import json
import re
import pickle
from tqdm.auto import tqdm
import unidecode

In [5]:
all_fever = pd.read_csv('all_fever_upd.csv')
all_fever.head()

,id,verifiable,label,claim,evidence,evidence_sources,NER,valid_link
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],...","{'Fox_Broadcasting_Company', 'Nikolaj_Coster-W...","['Nikolaj Coster-Waldau', 'Fox Broadcasting Co...",True
1,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[[174271, 187498, 'Roman_Atwood', 1]], [[1742...",{'Roman_Atwood'},['Roman Atwood'],True
2,214861,VERIFIABLE,SUPPORTS,"History of art includes architecture, dance, s...","[[[255136, 254645, 'History_of_art', 2]]]",{'History_of_art'},[],True
3,156709,VERIFIABLE,REFUTES,Adrienne Bailon is an accountant.,"[[[180804, 193183, 'Adrienne_Bailon', 0]]]",{'Adrienne_Bailon'},['Adrienne Bailon'],True
4,129629,VERIFIABLE,SUPPORTS,Homeland is an American television spy thrille...,"[[[151831, 166598, 'Homeland_-LRB-TV_series-RR...","{'Prisoners_of_War_(TV_series)', 'Homeland_(TV...","['American', 'Israeli', 'Prisoners of War']",True


## Loading and validating candidates: 

In [6]:
candidates_files = ['parsed_candidates10.pickle', 
                    'parsed_candidates_30.pickle', 
                    'parsed_candidates_50.pickle',
                    'parsed_candidates_spacy_10_en_core_web_sm.pickle', 
                    'parsed_candidates_spacy_30_en_core_web_sm.pickle', 
                    'parsed_candidates_spacy_50_en_core_web_sm.pickle',
                    'parsed_candidates_spacy_10_en_core_web_sm_SEPAR_noextend.pickle',
                    'parsed_candidates_spacy_10_en_core_web_sm_SEPAR_extend.pickle',
                    'trf_10_merge_noextend.pickle',
                    'parsed_candidates_spacy_trf_3_SEPAR_noextend.pickle',
                    'trf_5_SEPAR_noextend.pickle',
                    "trf_10_SEPAR_noextend.pickle",
                    'trf_10_SEPAR_extend.pickle',
                    'parsed_candidates_flair_10_simple.pickle',
                    'parsed_candidates_flair_3_SEPAR_notextend.pickle',
                    'parsed_candidates_flair_5_SEPAR_notextend.pickle',
                    'flair_10_SEPAR_noextend.pickle',
                    'flair_10_SEPAR_extend.pickle' # that should be changed
                    ]

In [10]:
from ast import literal_eval

res = 0

def calculate_acc(a,b):
    if type(a) is not set:
        a = literal_eval(a)
    f = 0
    for i in a:
        for j in b:
            if unidecode.unidecode(i) == unidecode.unidecode(j):
                f+=1
    return f

def calculate_recall(a,b):
    if type(a) is not set:
        a = literal_eval(a)
    f = 0
    for i in a:
        for j in b:
            if unidecode.unidecode(i) == unidecode.unidecode(j):
                f+=1
    return f/len(a)



def get_metrics(file):
    print('processing')
    name = file[:-7]
    with open(file, 'rb') as handle:
        precaclc = pickle.load(handle)['results']
        
    precaclc = pd.DataFrame(precaclc, columns = ['query', 'candidates'])

    all_fever['idd'] = np.arange(0, len(all_fever))

    res = all_fever.merge(precaclc, left_on='claim', right_on='query')
    res = res.drop('evidence', axis=1).drop_duplicates('idd')
    
    res = res[res.valid_link]
    
    acc = []
    for s1, s2 in tqdm(zip(res.evidence_sources, res.candidates)):
#         acc.append(calculate_acc(s1,s2)>0)
        acc.append(calculate_recall(s1,s2))
    
    n_c = res['candidates'].apply(len).mean()
        
    ### General accuracy of API n==10
    return {'name': name, 'n_matched': len(acc), 'accuracy': np.mean(acc), 'n_candidates':n_c}

In [11]:
results = [get_metrics(file) for file in candidates_files]

processing


0it [00:00, ?it/s]

processing


0it [00:00, ?it/s]

processing


...

0it [00:00, ?it/s]

processing


0it [00:00, ?it/s]

processing


0it [00:00, ?it/s]

In [12]:
pd.DataFrame(results)



,name,n_matched,accuracy,n_candidates
0,parsed_candidates10,117778,0.627848,9.112822
1,parsed_candidates_30,117778,0.645186,25.021982
2,parsed_candidates_50,117778,0.648865,39.157101
3,parsed_candidates_spacy_10_en_core_web_sm,117777,0.809724,15.333996
4,parsed_candidates_spacy_30_en_core_web_sm,117778,0.833226,44.022559
5,parsed_candidates_spacy_50_en_core_web_sm,117778,0.840023,70.676468
6,parsed_candidates_spacy_10_en_core_web_sm_SEPA...,117761,0.834486,10.117806
7,parsed_candidates_spacy_10_en_core_web_sm_SEPA...,117672,0.838657,10.111335
8,trf_10_merge_noextend,117778,0.826538,15.096886
9,parsed_candidates_spacy_trf_3_SEPAR_noextend,117778,0.874569,6.928985


## Looking on one specific case

In [67]:
# file = 'parsed_candidates_spacy.pickle'
file = 'parsed_candidates_flair_10_SEPAR_extend.pickle'

print('processing')
name = file[:-7]
with open(file, 'rb') as handle:
    precaclc = pickle.load(handle)['results']

precaclc = pd.DataFrame(precaclc, columns = ['query', 'candidates'])

all_fever['idd'] = np.arange(0, len(all_fever))

res = all_fever.merge(precaclc, left_on='claim', right_on='query')
res = res.drop('evidence', axis=1).drop_duplicates('idd')
res = res[res.valid_link]

acc = []
for s1, s2 in tqdm(zip(res.evidence_sources, res.candidates)):
    acc.append(calculate_acc(s1,s2)>0)


processing


In [91]:
res[~res['accuracy']][['claim', 'evidence_sources', 'candidates', 'valid_link']]

,claim,evidence_sources,candidates,valid_link
128,Furia is adapted from a short story by Anna Po...,{'Furia_(film)'},"{Anna_Politkovskaya, Furia}",True
141,The Challenge was a scripted show.,{'The_Challenge_(TV_series)'},"{The_Horror_Show_at_Extreme_Rules, NCIS_(TV_se...",True
153,"Charles, Prince of Wales is patron of numerous...","{'Charles,_Prince_of_Wales'}","{Prince_Harry,_Duke_of_Sussex, Royal_tours_of_...",True
156,Pope John Paul II co-stars many actors besides...,{'Pope_John_Paul_II_(miniseries)'},"{Legitimacy_(family_law), Romeo_and_Juliet, Ch...",True
178,Filmfare is about cheese.,{'Filmfare'},"{1991_in_music, Delhi_Belly_(film), Kiara_Adva...",True
...,...,...,...,...
153042,Sledgehammer made an appearance on Good Mornin...,{'Sledgehammer_(Fifth_Harmony_song)'},"{Janice_Rand, Jeffrey_Dahmer, Reflection_(Fift...",True
153049,The Quiet is only a television show.,{'The_Quiet'},"{Mehmet_Oz, List_of_British_television_program...",True
153051,The Hit was released on DVD.,{'The_Hit_(1984_film)'},"{Bret_""Hit_Man""_Hart:_The_Best_There_Is,_the_B...",True
153066,Hermit crabs are arachnids.,"{'Arachnid', 'Hermit_crab', 'Decapoda'}","{Invertebrate, Gill, Eurypterid, Seashell, Jur...",True
